In [233]:
"""
MOVING AVERAGE CROSSOVER STRATEGY USING THE S&P 500 AND 3 YEARS OF DATA. 

Here we are looking for more short term strategies. We use long < 365, short < 365.

First,  we use the backtesting library to find the best crossover strategy using  3 years of data.
Next, we confirm the results using our own backtesting and metrics.

The backtesting library selected short 1, long 11. It tested combinations using short = 1 to 365
and long = 1 to 365 with a step of 1. 

However, most of the outperformance appeared to be attributed to one trade (sell at the top of the coronavirus
market, buy at the bottom). The short 1, long 11 was the perfect strategy for a violent V shaped downdraft 
and updraft.  

So we did the analysis excluding the most recent bear market to see if that made a difference. The library 
tested 66066 combinations using short = 1, long = 365, with a step of 1. 

This time the backtesting library selected short 219, long 228, similar to the short 215, long 225 it found
using 20 years of data. This strategy was up 58.5% vs 50% for a buy and hold. 

So the very long SMAs seem to work over 3,5,and 20 years of data. However, over three years it generated
only 3 trades. To generate more trades we tested 19701 combinations between 1 and 200 with step 1 (still 
excluding the coronavirus bear market.)
 
This resulted in short = 2 and long = 10, so we circled right back to very short smas. This time the 
sma crossover strategy returned 49% compared to a buy and hold strategy at 51%. However, this generated
84 trades. Again, performance was helped by the sharp drawdowns in 2018 which the strategy responded
quickly to. So it appears that the short smas have merit even after excluding the coronavirus bear market.
Avg trade duration was 13 days so this is an active strategy.

Using 20 years of data and short = 2, long =10 produced terrible results (-68%). So this is not an
all-weather SMA strategy. Maybe it is more of a late bull market, high volatility market strategy where
one is concerned with sharp downdrafts and wants to exit as soon as possible while still trading the 
volatility. It may also be a strategy that works when the government is rolling out stimulus, creating
volatility. However, it still did not beat buy and hold even though it was selected as the best strategy
using SMAs below 200. Therefore, all the SMA strategies using SMAs below 200 must have underperformed
buy and hold. 

"""

import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
import alpaca_trade_api as tradeapi
import requests
import os 
from dotenv import load_dotenv
import plotly.express as px
import panel as pn
import hvplot
import hvplot.pandas

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from datetime import datetime, timedelta


pn.extension()

pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.width", 1000)

pd.set_option("display.max_rows", None, "display.max_columns", None)

%matplotlib inline

In [234]:
#load alpaca keys

load_dotenv()

alpaca_api_key = os.getenv("Api_key")
alpaca_secret_key = os.getenv("Secret_key")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')


type(alpaca_api_key)

str

In [235]:
#Use alpha vantage to download 20 years of data for the our selected ticker. 

ticker = 'SPY'

stock_data_df = api.alpha_vantage.historic_quotes(ticker, adjusted=True, output_format='pandas')
stock_data_df.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2020-07-02,314.2379,315.70,310.5962,312.19,312.19,69214995.0,0.0,1.0
2020-07-01,309.5700,311.85,309.0371,310.57,310.57,71910372.0,0.0,1.0
2020-06-30,303.9900,310.20,303.8200,308.36,308.36,112828251.0,0.0,1.0
2020-06-29,301.4100,304.61,298.9300,304.46,304.46,79411577.0,0.0,1.0
2020-06-26,306.1600,306.39,299.4200,300.05,300.05,127811745.0,0.0,1.0


In [236]:
#Clean data

#Sort earliest to latest.
stock_data_df.sort_index(inplace=True, ascending=True)

# Drop nulls
stock_data_df.dropna(inplace=True)

# drop duplicates
stock_data_df.drop_duplicates(inplace=True)

#count nulls 
stock_data_df.isnull().sum()

stock_data_df.head()



,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2000-07-03,145.4375,147.4375,145.1875,147.2812,100.6579,1436600.0,0.0,1.0
2000-07-05,146.3750,146.6562,144.3750,144.6250,98.8425,2748200.0,0.0,1.0
2000-07-06,144.9375,146.4687,144.2187,145.7500,99.6114,5963200.0,0.0,1.0
2000-07-07,146.6875,148.7812,146.2500,148.0937,101.2132,3034800.0,0.0,1.0
2000-07-10,147.8750,148.9062,147.5312,147.8437,101.0423,2816100.0,0.0,1.0


In [237]:
#Set up stock data for backtesting, needs to be OHLCV

stock_data_df.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
stock_data_df.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
stock_data_df.head()
#stock_data_df.to_csv('stock_data.csv')


,Open,High,Low,Close,Volume
date,,,,,
2000-07-03,145.4375,147.4375,145.1875,100.6579,1436600.0
2000-07-05,146.3750,146.6562,144.3750,98.8425,2748200.0
2000-07-06,144.9375,146.4687,144.2187,99.6114,5963200.0
2000-07-07,146.6875,148.7812,146.2500,101.2132,3034800.0
2000-07-10,147.8750,148.9062,147.5312,101.0423,2816100.0


In [238]:

#Slice out recent bear market
stock_data_df.drop(stock_data_df.tail(94).index,inplace=True)
stock_data_df.tail()

#Slice out last 3 years of data
stock_data_df = stock_data_df.iloc[-750:]

# #stock_data_df = stock_data_df.iloc[:200]
stock_data_df.head()

,Open,High,Low,Close,Volume
date,,,,,
2017-02-27,236.64,237.31,236.350,221.6470,56515440.0
2017-02-28,236.67,236.95,236.015,221.0487,96961938.0
2017-03-01,238.39,240.32,238.370,224.1428,149158170.0
2017-03-02,239.56,239.57,238.210,222.7313,70245978.0
2017-03-03,238.17,238.61,237.730,222.8715,81974300.0


In [239]:
len(stock_data_df)

750

In [240]:

#Function to do a backtest using Backtesting library. This function will find the best SMA cross strategy by analyzing
#different combinations of long short smas.

#Change the ranges in the stats for different scenarios. For example, range(5,365,5) means to test smas 
#between 5 and 365 days and use a 5 day step. 

def backtest():
    class SmaCross(Strategy):
        
    #n1 =short, n2 = long, they can be anything at the start of the optimization   
        n1=1
        n2=200
        
        def init(self):
            Close = self.data.Close
            self.ma1 = self.I(SMA, Close,self.n1)
            self.ma2 = self.I(SMA, Close,self.n2)

        def next(self):
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()

        
                
    bt_stock = Backtest(stock_data_df, SmaCross,
                              cash=10000, commission=.000)
  
    stats = bt_stock.optimize(n1=range(1, 200, 1),
                    n2=range(1, 200, 1),
                    maximize='Equity Final [$]',
                    constraint=lambda p: p.n1 < p.n2)
   

    bt_stock.run()
    #bt_spy_plot = bt_stock.plot()
    #return bt_stock_plot
    return stats
    #return bt_stock.run()
backtest()


C:\Users\costa\Anaconda3\envs\vizenv\lib\site-packages\ipykernel_launcher.py:33: UserWarning:

Searching best of 19701 configurations.



Start                      2017-02-27 00:00:00
End                        2020-02-19 00:00:00
Duration                    1087 days 00:00:00
Exposure [%]                           97.4241
Equity Final [$]                       14916.2
Equity Peak [$]                        15608.7
Return [%]                             49.1624
Buy & Hold Return [%]                  51.0475
Max. Drawdown [%]                     -19.2506
Avg. Drawdown [%]                     -5.50902
Max. Drawdown Duration       323 days 00:00:00
Avg. Drawdown Duration        36 days 00:00:00
# Trades                                    84
Win Rate [%]                           51.1905
Best Trade [%]                         11.2357
Worst Trade [%]                       -2.70926
Avg. Trade [%]                        0.509315
Max. Trade Duration           71 days 00:00:00
Avg. Trade Duration           13 days 00:00:00
Expectancy [%]                         1.50105
SQN                                    1.95582
Sharpe Ratio 

In [241]:
# To confirm the above results we will do our own tests. 

#To set up the crossover strategy select the one column we need, "Close", and set to dataframe
signals_df = stock_data_df['Close'].to_frame()

signals_df.head()


,Close
date,
2017-02-27,221.6470
2017-02-28,221.0487
2017-03-01,224.1428
2017-03-02,222.7313
2017-03-03,222.8715


In [242]:
#Generate signals and put in dataframe

# Set the short window and long windows
short_window = 2
long_window = 10

#Set names of the windows
short_name = "SMA2"
long_name = "SMA10"

# Generate the short and long moving averages 
signals_df[short_name] = signals_df["Close"].rolling(window=short_window).mean()
signals_df[long_name] = signals_df["Close"].rolling(window=long_window).mean()
signals_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the short SMA is under the long SMA, and
# where 1 is when the short SMA is higher (or crosses over) the long SMA
signals_df["Signal"][short_window:] = np.where(
    signals_df[short_name][short_window:] > signals_df[long_name][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Buy/Sell"] = signals_df["Signal"].diff()

signals_df.head()


,Close,SMA2,SMA10,Signal,Buy/Sell
date,,,,,
2017-02-27,221.6470,NaN,NaN,0.0,NaN
2017-02-28,221.0487,221.34785,NaN,0.0,0.0
2017-03-01,224.1428,222.59575,NaN,0.0,0.0
2017-03-02,222.7313,223.43705,NaN,0.0,0.0
2017-03-03,222.8715,222.80140,NaN,0.0,0.0


In [243]:
len(signals_df)

750

In [245]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Buy/Sell'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Buy/Sell'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# # Visualize close price for the investment
# security_close = signals_df[['Close']].hvplot(
#     line_color='lightgray',
#     ylabel='Price in $',
#     width=1000,
#     height=400
# )
# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='black',
    ylabel='Price in $',
    width=1000,
    height=400
)
# Visualize moving averages
moving_avgs = signals_df[['SMA2', 'SMA10']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .Scatter.Close.I  :Scatter   [date]   (Close)
   .Scatter.Close.II :Scatter   [date]   (Close)

In [171]:
signals_df.head()

,Close,SMA1,SMA11,Signal,Buy/Sell
date,,,,,
2017-02-27,221.6470,221.6470,NaN,0.0,NaN
2017-02-28,221.0487,221.0487,NaN,0.0,0.0
2017-03-01,224.1428,224.1428,NaN,0.0,0.0
2017-03-02,222.7313,222.7313,NaN,0.0,0.0
2017-03-03,222.8715,222.8715,NaN,0.0,0.0


In [172]:
# Add share and cash columns

signals_df["Shares"] = float(0)
signals_df["Portfolio Cash"] = float(100000)

#Set up starting cash variable for sma strategy and buy and hold strategy

cash = 100000
buy_and_hold_cash = 100000


#Iterate through each row, use combinations of signal and buy/sell points to set shares and portfolio cash for each row.

for index, row in signals_df.iterrows():
    
    #If at a buy point, calculate the number of shares using total cash available and set the cash to zero
    
    if row['Buy/Sell'] == 1.0:
       
        shares = cash/row['Close']
        row['Shares'] = shares
        row['Portfolio Cash']= 0 
       
    #If at sell point, calculate the cash generated from the sale (shares X closing price) and set the shares to zero, then
    #update the cash variable with the new cash balance'''
    
    elif row['Buy/Sell'] == -1.0:
        row['Portfolio Cash']= shares*row['Close']
        cash = row['Portfolio Cash']
        row['Shares']= 0
        
    #If we are not at a buy or sell point, look to signal column, if the signal is buy, then set cash to zero and update shares.
    # If the signal is sell, update the cash column using cash variable
    else:
     
        row['Buy/Sell'] == 0
        if row['Signal']== 1:
            row['Portfolio Cash'] = 0
            row["Shares"]= shares
        elif row['Signal'] == 0:
            row['Portfolio Cash'] = cash
        
# Add a Portfolio Total column
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + (signals_df['Shares']*signals_df['Close'])

# Calculate how many shares a buy and hold investor could buy at the start of the comparison. Use end of long window as start. 
buy_and_hold_shares = buy_and_hold_cash/signals_df['Close'].iloc[long_window-1]

# Add a buy and hold total column for comparison, set the first 225 (long window) to 100000 so both portfolios are at 100000 at the
#end of the long window calculation
signals_df["Buy&Hold Total"]= buy_and_hold_shares * signals_df['Close']
signals_df["Buy&Hold Total"][0:long_window-1] = buy_and_hold_cash      
        
# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()  
signals_df['Buy&Hold Daily Returns']= signals_df['Buy&Hold Total'].pct_change()

# Calculate the cumulative returns
signals_df['Portfolio Cum Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1
signals_df['Buy&Hold Cum Returns'] = (1 + signals_df['Buy&Hold Daily Returns']).cumprod() - 1

# Print the DataFrame
pd.set_option("display.max_rows", None, "display.max_columns", None)
signals_df.head(50)

,Close,SMA1,SMA11,Signal,Buy/Sell,Shares,Portfolio Cash,Portfolio Total,Buy&Hold Total,Portfolio Daily Returns,Buy&Hold Daily Returns,Portfolio Cum Returns,Buy&Hold Cum Returns
date,,,,,,,,,,,,,
2017-02-27,221.6470,221.6470,NaN,0.0,NaN,0.000000,100000.000000,100000.000000,100000.000000,NaN,NaN,NaN,NaN
2017-02-28,221.0487,221.0487,NaN,0.0,0.0,0.000000,100000.000000,100000.000000,100000.000000,0.000000,0.000000,0.000000,0.000000
2017-03-01,224.1428,224.1428,NaN,0.0,0.0,0.000000,100000.000000,100000.000000,100000.000000,0.000000,0.000000,0.000000,0.000000
2017-03-02,222.7313,222.7313,NaN,0.0,0.0,0.000000,100000.000000,100000.000000,100000.000000,0.000000,0.000000,0.000000,0.000000
2017-03-03,222.8715,222.8715,NaN,0.0,0.0,0.000000,100000.000000,100000.000000,100000.000000,0.000000,0.000000,0.000000,0.000000
2017-03-06,222.2078,222.2078,NaN,0.0,0.0,0.000000,100000.000000,100000.000000,100000.000000,0.000000,0.000000,0.000000,0.000000
2017-03-07,221.5441,221.5441,NaN,0.0,0.0,0.000000,100000.000000,100000.000000,100000.000000,0.000000,0.000000,0.000000,0.000000
2017-03-08,221.1328,221.1328,NaN,0.0,0.0,0.000000,100000.000000,100000.000000,100000.000000,0.000000,0.000000,0.000000,0.000000
2017-03-09,221.4133,221.4133,NaN,0.0,0.0,0.000000,100000.000000,100000.000000,100000.000000,0.000000,0.000000,0.000000,0.000000


In [174]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Buy/Sell'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Buy/Sell'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                    :Curve   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [date]   (Portfolio Total)

In [175]:
#Plot the SMA Strategy vs the Buy & Hold strategy

signals_df[["Buy&Hold Total", "Portfolio Total"]].hvplot()

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [68]:
#Prepare evaluation metrics dataframe

# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest', 'Buy&Hold']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` 
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,NaN,NaN
Cumulative Returns,NaN,NaN
Annual Volatility,NaN,NaN
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [69]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'][0] = signals_df['Portfolio Cum Returns'][-1]
portfolio_evaluation_df.loc['Cumulative Returns'][1] = signals_df['Buy&Hold Cum Returns'][-1]

portfolio_evaluation_df




,Backtest,Buy&Hold
Annual Return,NaN,NaN
Cumulative Returns,0.76793,0.628312
Annual Volatility,NaN,NaN
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [70]:
# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'][0] = (signals_df['Portfolio Daily Returns'].mean() * 252)
portfolio_evaluation_df.loc['Annual Return'][1] = (signals_df['Buy&Hold Daily Returns'].mean() * 252)


portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,0.128097,0.112889
Cumulative Returns,0.76793,0.628312
Annual Volatility,NaN,NaN
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [71]:
# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'][0] = (signals_df['Portfolio Daily Returns'].std() * np.sqrt(252))
portfolio_evaluation_df.loc['Annual Volatility'][1] = (signals_df['Buy&Hold Daily Returns'].std() * np.sqrt(252))
portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,0.128097,0.112889
Cumulative Returns,0.76793,0.628312
Annual Volatility,0.168485,0.175565
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [72]:
# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'][0] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df.loc['Sharpe Ratio'][1] = (
    signals_df['Buy&Hold Daily Returns'].mean() * 252) / (
    signals_df['Buy&Hold Daily Returns'].std() * np.sqrt(252)
)


portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,0.128097,0.112889
Cumulative Returns,0.76793,0.628312
Annual Volatility,0.168485,0.175565
Sharpe Ratio,0.760284,0.643004
Sortino Ratio,NaN,NaN


In [73]:
# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

sortino_ratio_df2 = signals_df[['Buy&Hold Daily Returns']].copy()
sortino_ratio_df2.loc[:,'Downside Returns1'] = 0

target = 0
mask = sortino_ratio_df2['Buy&Hold Daily Returns'] < target
sortino_ratio_df2.loc[mask, 'Downside Returns1'] = sortino_ratio_df2['Buy&Hold Daily Returns']**2


portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,0.128097,0.112889
Cumulative Returns,0.76793,0.628312
Annual Volatility,0.168485,0.175565
Sharpe Ratio,0.760284,0.643004
Sortino Ratio,NaN,NaN


In [74]:
# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'][0] = sortino_ratio

down_stdev = np.sqrt(sortino_ratio_df2['Downside Returns1'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df2['Buy&Hold Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'][1] = sortino_ratio


In [75]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


In [76]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Buy/Sell'] == 1:
        entry_date = index
        entry_portfolio_holding = row['Shares']*row['Close']
        share_size = row['Shares']
        entry_share_price = row['Close']

    elif row['Buy/Sell'] == -1:
        exit_date = index
        exit_portfolio_holding = row['Portfolio Total']
        exit_share_price = row['Close']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'SPY',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df
                
                

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,SPY,2016-07-11,2016-09-20,506.812831,197.3115,198.3307,100000.000000,100516.543638,516.543638
1,SPY,2016-09-29,2018-12-06,503.838283,199.5016,260.4992,100516.543638,131249.469701,30732.926063
2,SPY,2018-12-24,2019-06-11,576.725040,227.5772,281.7925,131249.469701,162516.790744,31267.321044


In [77]:
#Create plots

price_df = signals_df[['Close', 'SMA220', 'SMA230']]
price_chart = price_df.hvplot.line()
price_chart.opts(xaxis=None)

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [78]:
portfolio_evaluation_df.reset_index(inplace=True)
portfolio_evaluation_table = portfolio_evaluation_df.hvplot.table()
portfolio_evaluation_table

:Table   [index,Backtest,Buy&Hold]

In [79]:
trade_evaluation_table = trade_evaluation_df.hvplot.table()
trade_evaluation_table

:Table   [Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss]

In [80]:
#Build dashboard with panel

# Create rows
price_chart_row = pn.Row(price_chart)
portfolio_evaluation_row = pn.Row(portfolio_evaluation_table)
trade_evaluation_row = pn.Row(trade_evaluation_table)

# Create columns
portfolio_column = pn.Column('# Portfolio Evaluation Metrics', price_chart_row, portfolio_evaluation_row)
trade_column = pn.Column('# Trade Evaluation Metrics', trade_evaluation_row)

# Create tabs
trading_dashboard = pn.Tabs(
    ("Portfolio Metrics", portfolio_column),
    ("Trade Metrics", trade_column)
)

In [81]:
trading_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(NdOverlay)
        [2] Row
            [0] HoloViews(Table)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(Table)